# Forecast Random Forest

1st August 2022

Iterates on CNN2 to include all the geostationary data.

In [22]:
# Libraries
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import numpy as np
import copy
import os
import time
#from google.cloud import storage
from datetime import timedelta, timezone, datetime
import zarr
#from google.cloud import storage
import pickle5
import pickle
import pandas as pd
from datetime import datetime, timedelta
import fsspec
#from gcsfs import GCSFileSystem
from tqdm import tqdm, notebook
#from google.cloud import bigquery
import seaborn as sns
import jax
import jax.numpy as jnp
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix
import scipy.special as scp
from sklearn.ensemble import RandomForestClassifier

from itertools import permutations, combinations
from scipy.stats import norm
import math


%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
from google.cloud import bigquery

In [24]:
os.chdir('../')
os.getcwd()

'/home/emiliano/Documents/FDL'

In [29]:
os.chdir('./pyrocast_clean/')

In [30]:
os.getcwd()

'/home/emiliano/Documents/FDL/pyrocast_clean'

In [45]:
# Custom libraries
import sys
sys.path.insert(1,'/utils/')

import utils.dataprep as dp
from hsic import *
#from rf import *
from icp import *
#import icp_nets as icpn

## New Load Data

### 1.5 | Split data

In [65]:
ML_ready_root = "../data/"
file = "Himawari_lazyload_dataset_keys_prediction_v1.pkl"

keys = pickle5.load( open( ML_ready_root+file, "rb" ) )
keys_image, keys_era5 = keys

In [66]:
keys = keys_era5

In [67]:
k0 = keys[1]["flag_then"][0]
k0

[True, True, False, False, True, True]

In [68]:
k0[1:len(k0)]

[True, False, False, True, True]

In [69]:
k0[0:(len(k0)-1)]

[True, True, False, False, True]

In [70]:
[k0[0]]+(np.array(k0[1:len(k0)])*np.logical_not(k0[0:(len(k0)-1)])).tolist()

[True, False, False, False, True, False]

In [71]:
def correctFlags(ks):
    return [ks[0]]+(np.array(ks[1:len(ks)])*np.logical_not(ks[0:(len(ks)-1)])).tolist()

In [72]:
i = 0
correctedFlags = [correctFlags(keys[1]["flag_then"][i]) for i in range(len(keys[1]["flag_then"]))]

In [73]:
np.sum(dp.flatten(keys[1]["flag_then"]))

1167

In [74]:
np.sum(dp.flatten(correctedFlags))

447

In [75]:
len(dp.flatten(correctedFlags))

6919

In [76]:
i = 0
indxKeep, = np.where([dp.flatten(correctedFlags)[i]==dp.flatten(keys[1]["flag_then"])[i] for i in range(len(dp.flatten(correctedFlags)))])

In [77]:
indxKeep.shape

(6199,)

In [78]:
envVars = np.load(ML_ready_root+'env_era5_pred_v1.npy')
envVars.shape

FileNotFoundError: [Errno 2] No such file or directory: '../data/env_era5_pred_v1.npy'

In [ ]:
envVars[0:5,]


Meteorological Era 5 variables

| Variable index | Short name | Long name |
| -------------- | ---------- | --------- |
| 0  | u10  | 10m u component of wind | 
| 1  | v10  | 10m v component of wind |
| 2  | fg10 | 10m wind gust since previous post processing |
| 3  | blh  | boundary layer height |
| 4  | cape | convective available potential energy |
| 5  | cin  | convective inhibition |
| 6  | z    | geopotential |
| 7  | slhf | surface latent heat flux |
| 8  | sshf | surface sensible heat flux |
| 9 | w    | surface vertical velocity |
| 10 | u    | u component of wind at 250hPa |
| 11 | v    | v component of wind at 250hPa |
| 12 | cvh  | fraction of high vegetation |
| 13 | cvl  | fraction of low vegetation |
| 14 | tvh  | type of high vegetation |
| 15 | tvl  | type of low vegetation |
| 16 | r650 | relative humidity at 650hPa |
| 17 | r750 | relative humidity at 750hPa |
| 18 | r850 | relative humidity at 850hPa |

In [ ]:
#cube = np.load(ML_ready_root+'x_image_det_v1.npy')
#labels = np.load(ML_ready_root+'y_image_det_v1.npy')
#cube = np.load(ML_ready_root+'x_image_pred_v1.npy')
#labels = np.load(ML_ready_root+'y_image_pred_v1.npy')

#cube = np.load(ML_ready_root+'xFeat_era5_3vs_pred_v1.npy')
#labels = np.load(ML_ready_root+'y_era5_3vs_pred_v1.npy')
#cube = np.load(ML_ready_root+'xFeat_era5_19vs_pred_v1.npy')
cube = np.load(ML_ready_root+'xFeat_era5_25vs_pred_v2.npy')
labels = np.load(ML_ready_root+'y_era5_19vs_pred_v1.npy')

#cube = np.load(ML_ready_root+'xFeat_era5_3vs_pred_oracle_v1.npy')
#labels = np.load(ML_ready_root+'y_era5_3vs_pred_oracle_v1.npy')
#cube = np.load(ML_ready_root+'xFeat_era5_19vs_pred_oracle_v1.npy')
#labels = np.load(ML_ready_root+'y_era5_19vs_pred_oracle_v1.npy')

#cube = np.load(ML_ready_root+'xFeat_era5_3vs_det_v1.npy')
#labels = np.load(ML_ready_root+'y_era5_3vs_det_v1.npy')
#cube = np.load(ML_ready_root+'xFeat_era5_19vs_det_v1.npy')
#labels = np.load(ML_ready_root+'y_era5_19vs_det_v1.npy')

In [ ]:
def allZeros(x):
    return np.all(x==0)
indxZeros = np.where(np.apply_along_axis(allZeros,0,cube))
indxZeros

In [ ]:
#cube = cube[:,0:66]
#cube = cube[:,66:99]
#cube = cube[:,66:275]

In [ ]:
cube.shape

In [ ]:
labels.shape

In [ ]:
np.sum(labels)

In [ ]:
labels

In [ ]:
np.sum(np.isnan(cube))

In [ ]:
11*9

In [ ]:
11*25

In [ ]:
11*6

In [ ]:
def norml(x):
    return (x-np.min(x))/(np.max(x)-np.min(x))


In [ ]:
def norml2(x):
    den = (np.nanmax(x)-np.nanmin(x))
    if den>0:
        #print("ya")
        res = (x-np.nanmin(x))/den
    else: 
        print("bla")
        res = x
    return res


In [ ]:
def norml3(x,i):
    #print(i)
    minX = np.quantile(x[x>0], 0.01)
    maxX = np.quantile(x[x>0], 0.99)
    res = x
    res[x<=minX]=minX
    res[x>=maxX]=maxX
    res = (res-minX)/(maxX-minX)
    
    return  res

In [ ]:
np.max(cube)

In [ ]:
#np.quantile(cube[:,145], 0.01)

In [ ]:
#sns.distplot(cube[:,145])

In [ ]:
cube.shape

In [ ]:
i = 0
cube = np.array([norml2(cube[:,i]) for i in range(cube.shape[1])])
cube = np.transpose(cube,[1,0])
cube.shape


In [ ]:
cube[np.isnan(cube)] = 0
np.sum(np.isnan(cube))

In [ ]:
np.prod(cube.shape)

In [ ]:
np.sum(np.isinf(cube))

In [ ]:
25*11

In [ ]:
#cube = np.transpose(np.array([norml(cube[:,i,:,:]) for i in range(cube.shape[1])]),[1,0,2,3])

In [ ]:
event_ids = np.array(keys[1]["event_id"])[np.array(keys[0]["level1_key"])]

In [ ]:
len(dp.flatten(keys[1]["full_flag_now"]))

In [ ]:
len(dp.flatten(keys[1]["flag_then"]))

In [ ]:
repos = "../data/"
file = "wildfires.pkl"
wildfires = pickle5.load( open( repos+file, "rb" ) )
wildfires.loc[pd.isnull(wildfires.wildfire_id),"wildfire_id"] = wildfires.loc[pd.isnull(wildfires.wildfire_id),"pyroCb_id"]

In [ ]:
wildfires.head()

In [ ]:
pyrocb_id = [int(ei.split("_")[0]) for ei in event_ids.tolist()]

In [ ]:
event_df = pd.DataFrame({"pyrocb_id":pyrocb_id, "event_id":event_ids, "full_flag_now":dp.flatten(keys[1]["full_flag_now"]), "flag_then":dp.flatten(keys[1]["flag_then"])})
event_df.head()

In [ ]:
event_df = event_df.merge(wildfires[["pyroCb_id","wildfire_id"]], how="left", left_on="pyrocb_id", right_on="pyroCb_id")
event_df.shape

In [ ]:
event_df = pd.concat([event_df, pd.DataFrame(envVars,columns=["lon","lat","date","year","sat"])], axis=1)
event_df.shape

In [ ]:
indx, = np.where(event_df.full_flag_now<4)
indx.shape

In [60]:
# exlcude pyrocbs now
#cube = cube[indx,]
#labels = labels[indx,]
#envVars = envVars[indx,]
#event_df = event_df.iloc[indx]
cube.shape

NameError: name 'cube' is not defined

In [61]:
seg_base = event_df.drop_duplicates('wildfire_id')
seg_base.shape

NameError: name 'event_df' is not defined

In [62]:
# GOES = 1, HIMAWARI = 0
seg_base[["pyrocb_id","sat"]].groupby(["sat"]).count()

NameError: name 'seg_base' is not defined

In [63]:
# GOES = 1, HIMAWARI = 0
seg_base[["pyrocb_id","sat","year"]].groupby(["sat","year"]).count()

NameError: name 'seg_base' is not defined

In [64]:
seg_base[["sat","lon","lat","date"]].head()

NameError: name 'seg_base' is not defined

In [ ]:
def stdze(x):
    return (x-np.mean(x))/np.std(x)

In [ ]:
def addCluster(seg_base, num_clusters,seed):
    np.random.seed(seed=seed)
    rnds = np.floor(np.arange(0,seg_base.shape[0])/17).astype(int)
    #seg_base2 = seg_base[["sat","lon","lat","date"]].to_numpy()
    seg_base2 = seg_base[["sat","lon","lat"]].to_numpy()
    seg_base2 = np.array([stdze(seg_base2[:,i]) for i in range(seg_base2.shape[1])]).T
    N = seg_base2.shape[0]
    cluster_size = N//num_clusters
    print("cluster size: ", cluster_size)
    labs = -1*np.ones(N)
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(seg_base2)
    grps = kmeans.labels_
    ids, cnt = np.unique(grps, return_counts=True)
    print(pd.DataFrame({"cluster":ids,"count":cnt}))
    o = np.argsort(cnt)
    DistMat = kmeans.fit_transform(seg_base2)
    
    # initialize
    indx_obs = np.arange(seg_base2.shape[0]).tolist()
    indx_cluster = ids[o][::-1].tolist()
    
    # loop through clusters from smallest to largest
    for i in range(len(indx_obs)):
        #print("i: ", i)
        o2 = np.argsort(DistMat[indx_obs,indx_cluster[0]])
        indx_lab = np.array(indx_obs)[o2[0]]
        labs[indx_lab] = indx_cluster[0]
        indx_cluster.pop(0)
        if len(indx_cluster)==0:
            indx_cluster = ids[o][::-1].tolist()
        indx_obs = list(set(indx_obs).difference([indx_lab]))
    
    
    #seg_base["cluster"]=np.array(labs).astype(int)
    seg_base["cluster_regional"]=np.array(grps).astype(int)
    seg_base["cluster_random"]=np.array(rnds).astype(int)
    return seg_base, seg_base2

In [ ]:
rand = np.floor(np.arange(0,seg_base.shape[0])/17).astype(int)
cnt, uni = np.unique(rand, return_counts=True)
{"cnt":cnt, "uni":uni}

In [ ]:
83/5

In [ ]:
num_clusters=5
seed = 123456
seg_base, X = addCluster(seg_base, num_clusters=num_clusters, seed=seed)

In [ ]:
results_root = "..//results/"
version = "v0"
file = "seg_base"
fileRes = results_root+file+"_"+version+".pkl" 
fileRes

In [ ]:
with open(fileRes, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(seg_base, output, pickle.HIGHEST_PROTOCOL)


In [ ]:
cluster_var = "cluster_random"
#cluster_var = "cluster_regional"
seg_base[["pyrocb_id",cluster_var]].groupby(cluster_var).count()

In [ ]:
metrics.silhouette_score(X, seg_base[cluster_var], metric='euclidean')

In [ ]:
N = seg_base.shape[0]
cluster_size = N//num_clusters

rand_labs = np.arange(seg_base.shape[0])
rand_labs = (rand_labs//cluster_size)%num_clusters
indx = np.random.choice(seg_base.shape[0], size=seg_base.shape[0], replace=False)
rand_labs = rand_labs[indx]

In [ ]:
metrics.silhouette_score(X, rand_labs, metric='euclidean')

In [ ]:
seg_base.head()

In [ ]:
results_root = "/home/jupyter/ML_ready/results/"
typee = cluster_var
fileRes = results_root+typee+"_clustering.png"
fileRes

In [ ]:
i = 0
fig, ax = plt.subplots(1, 2, figsize=(6, 6))  # setup the plot
for i in np.unique(seg_base[cluster_var]):
    scat1 = ax[0].scatter(seg_base.loc[seg_base[cluster_var]==i]["lon"], seg_base.loc[seg_base[cluster_var]==i]["lat"], label=i)#, s=date_norm
    scat1 = ax[1].scatter(seg_base.loc[seg_base[cluster_var]==i]["lon"], seg_base.loc[seg_base[cluster_var]==i]["lat"], label=i)
ax[0].set_xlim([np.min(seg_base["lon"])-2, -100])
ax[0].set_ylim([30,np.max(seg_base["lat"])+2])

ax[1].set_xlim([115,np.max(seg_base["lon"])+1])
ax[1].set_ylim([np.min(seg_base["lat"])-2, -10])


fig.text(0.5, 0.04, 'longitude', ha='center')
fig.text(0.04, 0.5, 'latitude', va='center', rotation='vertical')



ax[0].set_title('North America')
ax[1].set_title('Australia')
ax[1].legend(bbox_to_anchor=(1.32, 1.0))
#plt.savefig(fileRes, format='png', dpi=300)


In [ ]:
event_df = event_df.merge(seg_base[["wildfire_id","cluster_regional","cluster_random"]], how="left",on="wildfire_id")

In [ ]:
event_df.head()

In [ ]:
event_df[["pyrocb_id",cluster_var]].groupby(cluster_var).count()

In [ ]:
event_df.shape

In [ ]:
#file = "/home/jupyter/elevation/alwdgg.tif"
#da = xr.open_rasterio(file)
#alt = [da.interp(x=event_df.iloc[i].lon, y = event_df.iloc[i].lat, method="linear").values[0] for i in range(event_df.shape[0])]

In [ ]:
#event_df["altitude"] = alt
#def meanAlt(x):
#    return np.mean(x.altitude)
#event_df[["pyrocb_id","cluster","altitude"]].groupby("cluster").apply(meanAlt)

In [ ]:
print(envVars.shape)
print(cube.shape)
print(labels.shape)
print(len(keys[0]["level1_key"]))

In [ ]:
plt.plot(envVars[:,2])

In [ ]:
mu_env = [np.mean(envVars[:,i]) for i in range(envVars.shape[1])]
sigma_env = [np.std(envVars[:,i]) for i in range(envVars.shape[1])]

In [ ]:
mu_env

In [ ]:
sigma_env

In [ ]:
satellite_list = dp.flatten(keys[1]["satellite"])

In [ ]:
satellite_list = np.array(satellite_list)

In [ ]:
event_ids = np.array(keys[1]["event_id"])[np.array(keys[0]["level1_key"])]

In [ ]:
aux = [(e,v) for (e,v) in zip(event_ids.astype(float), envVars[:,2])]

In [ ]:
aux = np.array( aux, dtype=[('x', 'float'), ('y', 'float')])
aux.shape


In [ ]:
o = np.argsort(aux, order=('x','y'))


In [ ]:
plt.plot(o)

In [ ]:
event_ids[o]

In [ ]:
envVars[o,2]

In [ ]:
#o = np.argsort(envVars[:,2])
#envVars = envVars[o,:]
#cube = cube[o,:]
#labels = labels[o]
#satellite_list = satellite_list[o]
#event_df= event_df.iloc[o]

In [ ]:

#envVars = envVars[indxKeep,:]
#cube = cube[indxKeep,:]
#labels = labels[indxKeep]
#satellite_list = satellite_list[indxKeep]
#event_df= event_df.iloc[indxKeep]

In [ ]:
cube.shape

In [ ]:
cube.shape

In [ ]:
event_df.shape

In [ ]:
np.sum(event_df.flag_then)

In [ ]:
np.sum(labels)

In [ ]:
event_df[["flag_then",cluster_var,"pyrocb_id"]].groupby([cluster_var,"flag_then"]).count()

In [ ]:
event_df[["flag_then","full_flag_now","pyrocb_id"]].groupby(["full_flag_now","flag_then"]).count()

In [ ]:
# cluster, wildfire_id, red_flag_now, flag_then

In [ ]:
event_df[["pyrocb_id","full_flag_now"]].groupby("full_flag_now").count()

In [ ]:
mydict = {0:"none", 1:"none", 2:"conv", 3:"deep conv",4:"pyrocb"}

In [ ]:
event_df["red_flag_now"] =event_df["full_flag_now"].replace(mydict)
event_df[["pyrocb_id","red_flag_now"]].groupby("red_flag_now").count()

In [ ]:
event_df.head()

In [ ]:
#cluster = 0
#event_df_clus = event_df.loc[event_df.cluster==cluster]
tab = event_df[["cluster_regional","cluster_random","pyrocb_id","wildfire_id","red_flag_now","flag_then"]].groupby([cluster_var,"wildfire_id","red_flag_now","flag_then"]).count()

In [ ]:
np.all(tab.to_numpy()>0)

In [ ]:
tab = tab.reset_index().rename(columns={"pyrocb_id":"freq"})

In [ ]:
np.max(tab.freq)

In [ ]:
np.max(tab.freq)*tab.shape[0]

In [ ]:
tab["weight"]= 1/tab.freq

In [ ]:
tab.head()

In [ ]:
event_df = event_df.merge(tab[["cluster_regional","cluster_random","wildfire_id","red_flag_now","flag_then","weight"]], how="left", on=["cluster_regional","cluster_random","wildfire_id","red_flag_now","flag_then"])

In [ ]:
mu = [np.mean(cube[:,i]) for i in range(cube.shape[1])]
sigma = [np.std(cube[:,i]) for i in range(cube.shape[1])]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#X_train, X_val, y_train, y_val = train_test_split(big_datacube, flag_list, test_size=0.20, random_state=42, stratify=None, shuffle=False)
#y_val.shape

In [ ]:
cutoff = int(envVars.shape[0]*0.8)
cutoff

In [ ]:
cube.shape

In [ ]:
np.sum(np.isnan(cube))

In [ ]:
# By satellite and day
#indx1,  = np.where(satellite_list != "Himawari")
#num_rem = int(cube.shape[0]*0.2)-indx1.shape[0]
#print(num_rem )
#indx2, = np.where(satellite_list == "Himawari")
#indx2 = indx2[0:num_rem]
#indx_val = np.hstack([indx1, indx2])
#indx_val.shape

In [ ]:
# by event_id and date
#indx_val = np.arange(cutoff, envVars.shape[0])
#indx_val

In [ ]:
#by cluster
cluster = 1
indx_val, =  np.where(event_df[cluster_var]==cluster)

In [ ]:
# by event_id: choose until you breach 20%

In [ ]:
indx_train = np.array(list(set(np.arange(0, cube.shape[0])).difference(indx_val)))

In [ ]:
cube.shape

In [ ]:
X_train = cube[indx_train,:]
X_val = cube[indx_val,:]
y_train = labels[indx_train]
y_val = labels[indx_val]
Env_train = envVars[indx_train,3:5]
Env_val = envVars[indx_val,3:5]
y_val.shape

In [ ]:
event_df.shape

In [ ]:
event_df_train = event_df.iloc[indx_train]
event_df_val = event_df.iloc[indx_val]

In [ ]:
envVars[0:10,]

In [ ]:
#met_varss = ["u10", "v10", "fg10", "blh", "cape", "cin", "z", "slhf", "sshf", "w", "u", "v", "cvh", "cvl", "tvh", "tvl", "r650", "r750", "r850"]
met_varss = ["u10", "v10", "fg10", "blh", "cape", "cin", "z", "slhf", "sshf", "w", "u", "v", "cvh", "cvl", "tvh", "tvl", "r650", "r750", "r850","uv10", "uv250", "alt", "typeH", "typeL"]
len(met_varss)

In [ ]:
img_varss = ["ch1","ch2","ch3","ch4","ch5","ch6"]
varss = img_varss+met_varss

In [ ]:
posts =np.arange(0,cube.shape[1]+1-10, 11)
posts = np.array(posts.tolist()+[posts[posts.shape[0]-1]+4,posts[posts.shape[0]-1]+10])
posts

# ICP

In [ ]:
print(varss)
print(len(varss))
indxIncl = [10,21]
indxIncl = [0,4,5,13,16,17,18]
#indxIncl = [29]
indxIncl = np.arange(len(varss)).tolist()
indxIncl 
np.array(varss)[indxIncl].shape

In [ ]:
i = 0
envv = np.array([(event_df["cluster_regional"]==i)*1 for i in np.arange(5)]).T
envv.shape

In [ ]:
envVars.shape

In [ ]:
30*11

In [ ]:
event_df.head()

## Conditional independence test Y indep E | X based on delong test for diff of AUC and Random Forests

### Obtain out-of-sample probability of ocurrence using random forest with an without environment E variables

In [ ]:

start_time = time.time()
y_pred_noE = getTestProbsWrapper(cube, labels, event_df, cluster_var, indxIncl, posts, numTrees=100)
y_pred_E = getTestProbsWrapper(cube, labels, event_df, cluster_var, indxIncl, posts, numTrees=100, envVar=envVars[:,0:3])
elapsed_time = time.time() - start_time
print("elapsed time: ", elapsed_time) # 30 secs for 5 folds, 10 combs each



In [ ]:
4000*30/60/60 # hours

In [ ]:
type(indxIncl)

In [ ]:
roc_auc_score(labels, y_pred_noE)

In [ ]:
roc_auc_score(labels, y_pred_E)

In [ ]:
elapsed_time/60 #3.4 mins for 500 trees, 41 secs for 100 trees

In [ ]:
np.sum(np.arange(len(varss))+1)*elapsed_time/60/60 #5 hours for greedy algo

In [ ]:
len(varss)

In [ ]:
varss

In [ ]:

def comb(n,p):
    return math.factorial(n)/(math.factorial(n-p)*math.factorial(p))

In [ ]:
comb(30,10)

In [ ]:
np.sum([comb(len(varss)-2,i) for i in range(len(varss)-2)])
#1,073,741,823

### given proabilities of pyroCb with and without using environment E variables obtain hypothesis test

In [ ]:
equalAUC_hypTest(y_pred_noE, y_pred_E, labels)

In [ ]:
type(labels)

In [ ]:
#getHypWrapper(cube, labels, event_df, cluster_var, indxIncl, posts, envVar=envv)

In [ ]:
range(len(indxIncl))

In [ ]:
#indxIncl = [10,21]
#indxIncl = [29]
indxIncl = np.arange(len(varss)).tolist()
indxIncl.pop(21)
indxIncl.pop(20)
np.array(varss)[indxIncl].shape


In [ ]:
envVars[:,0:3].shape

In [ ]:
envVars[0:5,:]

In [ ]:
def allZeros(x):
    return np.all(x==0)

indxZeros, = np.where(np.apply_along_axis(allZeros, 0, cube))
indxZeros

In [ ]:
cube.shape

### obtain conditional idependence test statistic and p-values where S is all variables in cube

In [ ]:
start_time = time.time()
print(getHypWrapper(cube, labels, envVars[:,0:3], event_df, cluster_var, [], posts, numTrees=100))
elapsed_time = time.time() - start_time
print("elapsed time: ", elapsed_time) # 30 secs for 5 folds, 10 combs each



## Greedy ICP algorithm

In [ ]:
varss  = img_varss+met_varss
indxIncl = np.arange(len(varss)).tolist()
cluster_var = "cluster_random"

In [ ]:
posts =np.arange(0,cube.shape[1]+1-10, 11)
posts = np.array(posts.tolist()+[posts[posts.shape[0]-1]+4,posts[posts.shape[0]-1]+10])
posts

In [ ]:
start_time = time.time()
#res = greedyICP(cube, labels, envVars[:,0:3], event_df, cluster_var, indxIncl, posts, varss, numTrees=100, pvalTest="pval_1tail")
elapsed_time = time.time() - start_time
print("elapsed time: ", elapsed_time) # 30 secs for 5 folds, 10 combs each




In [ ]:
type(posts)

In [ ]:
results_root = "/home/emiliano/Documents/FDL/clean_code/results/"
version = "v_1Tail_0"
file = "greedyICP"
fileRes = results_root+file+"_"+version+"_pvals.pkl" 



In [ ]:
#with open(fileRes, 'wb') as output:  # Overwrites any existing file.
#        pickle.dump(res, output, pickle.HIGHEST_PROTOCOL)


In [ ]:
res = pickle5.load( open( fileRes, "rb" ) )

In [ ]:
res["auc_E"][len(res["auc_E"])-1] = np.nan
res["auc_noE"][len(res["auc_noE"])-1] = np.nan

In [ ]:
res

In [ ]:
res.keys()

In [ ]:
plt.scatter(np.array(varss)[res["indxVar"]], res["pval"])

In [ ]:
plt.scatter(np.array(varss)[res["indxVar"]], np.log(res["pval"]))

## Get AUC for model with novars and with only E vars

### Get figure 2 from paper

In [ ]:
df = pd.DataFrame({"var":np.array(varss)[res["indxVar"]], "pval":res["pval"], "auc_noE":res["auc_noE"], "auc_E":res["auc_E"]})
df.head()

In [ ]:
ML_ready_root

### Figure 2a

In [ ]:
df2 = df.copy()
df2["dummy"] = 1
g = sns.stripplot(data=df2, x="var", y="pval", hue="dummy",size=10)
plt.xticks(rotation=90)
plt.xticks(size=12)
plt.xlabel("excluded variable",size=20)
plt.ylabel("p-value",size=20)
#g.set(xlabel='excluded variable', ylabel='p-value')
#plt.legend(bbox_to_anchor=(1.25, 1),borderaxespad=0)
plt.legend([],[], frameon=False)
#plt.savefig(ML_ready_root+"results/greedyICP_pval.png", format='png', dpi=300)

In [ ]:
df  =pd.melt(df, id_vars="var")
df.head()

### Figure 2b

In [ ]:
g = sns.stripplot(data=df.loc[df.variable!="pval"], x="var", y="value", hue="variable", size=8)
plt.xticks(rotation=90)
plt.xticks(size=12)
plt.xlabel("excluded variable",size=20)
plt.ylabel("auc",size=20)
plt.setp(g.get_legend().get_texts(), fontsize='18') # for legend text
plt.setp(g.get_legend().get_title(), fontsize='20') # for legend title
#g.set(xlabel='excluded variable', ylabel='p-value')
#plt.legend(bbox_to_anchor=(1.25, 1),borderaxespad=0)
#plt.savefig(ML_ready_root+"results/greedyICP_auc.png", format='png', dpi=300)

In [ ]:
len(res["indxVar"])

In [ ]:
np.sum([comb(12,i) for i in range(6,12)])
# 793

In [ ]:
elapsed_time*2500/60/60 #5 hours

In [ ]:
np.array(varss)[res["indxVar"]]

In [ ]:
np.array(varss)[res["indxVar"]]

In [ ]:
varss

In [ ]:
np.array(varss)[res["indxVar"][::-1][0:11]]

In [ ]:
indxIncl = res["indxVar"][::-1][0:11]
print(len(indxIncl))
np.array(varss)[indxIncl]

In [ ]:
[i for i in range(8,12)]

In [ ]:
np.array(varss)[res["indxVar"]]

## Run exhaustive ICP on all combinations of size 8 or larger from 12 variables selected using greedy ICP

In [ ]:
#varsSelec = ['v10','v','ch1','alt','sshf','u10','ch6','z','r650','cape','ch5','r850']#
varsSelec = ['alt', 'cape', 'ch6', 'v', 'r850', 'sshf', 'u10', 'ch1', 'z','uv10', 'blh']

In [ ]:
i = 0
indxIncl = np.sort([findIndxAux(varss, varsSelec, i) for i in range(len(varsSelec))]).tolist()

In [ ]:
i = 8
combos = dp.flatten([list(combinations(np.array(varss)[indxIncl], i)) for i in range(8,12)])
#combos = dp.flatten([list(combinations(np.array(varss)[indxIncl], i)) for i in range(4,12)])

In [ ]:
len(combos)

In [ ]:
combos = [list(c) for c in combos]

In [ ]:
start_time = time.time()
#res_ex = exhaustiveICP(cube, labels, envVars[:,0:3], event_df, cluster_var, varss, combos, posts, numTrees=100)
elapsed_time = time.time() - start_time
print("elapsed time: ", elapsed_time) # 30 secs for 5 folds, 10 combs each


In [ ]:
results_root = "/home/emiliano/Documents/FDL/clean_code/results/"
version = "v_1Tail_0"
file = "exhaustiveICP_8_11"
fileRes = results_root+file+"_"+version+"_pvals.pkl" 

In [ ]:
#results_root = "/home/jupyter/ML_ready/results/"
#version = "v_1Tail_0"
#file = "exhaustiveICP_8_11"
#fileRes = results_root+file+"_"+version+"_pvals.pkl" 

In [ ]:
#with open(fileRes, 'wb') as output:  # Overwrites any existing file.
#        pickle.dump(res_ex, output, pickle.HIGHEST_PROTOCOL)


In [ ]:
res_ex = pickle5.load( open( fileRes, "rb" ) )

In [ ]:
len(res_ex)

In [ ]:
sig = 0.05
indxCombos, = np.where([r["pval_1tail"]>sig for r in res_ex])

In [ ]:
combosPass = [combos[i] for i in indxCombos]
len(combosPass)

In [ ]:
combosPass

In [ ]:
len(combosPass)

In [ ]:
len(combos)

In [ ]:
np.array(varss)[indxIncl]

In [ ]:
ids, cnts = np.unique(dp.flatten(combosPass), return_counts=True)
df_freq = pd.DataFrame({"var":ids, "freq":cnts})
df_freq

In [ ]:
df_freq.freq

In [ ]:
df_freq2 = df_freq.copy()
df_freq2["dummy"] = 1


sns.barplot(data=df_freq2, x="var", y="freq", hue="dummy")
plt.xticks(rotation=90)
plt.xticks(size=12)
plt.xlabel("variable",size=20)
plt.ylabel("frequency",size=20)
plt.legend([],[], frameon=False)


#plt.savefig(ML_ready_root+"results/accepted_sets_freq.png", format='png', dpi=300)

In [ ]:
cmb = combosPass[0]
for i in range(1,len(combosPass)):
    cmb = set(cmb).intersection(combosPass[i])
list(cmb)

In [ ]:
results_root = "/home/emiliano/Documents/FDL/clean_code/results/"
version = "v_1Tail_0"
file = "exhaustiveICP_8_11"
fileRes = results_root+file+"_"+version+"_accepted_sets.json" 
fileRes

In [ ]:
import json
#with open(fileRes, 'w') as f:
#    json.dump(combosPass, f)

## Appendix A.2 analysis: cluster variables and apply exhaustive ICP on clusters

In [ ]:
maxSmpl = 100
smpl = np.random.choice(cube.shape[0], size=maxSmpl, replace=False)[:,None]

In [ ]:
np.array(varss)[indxIncl]

In [ ]:
posts

In [ ]:
indxs = [np.arange(posts[indxIncl[i]],posts[indxIncl[i]+1]) for i in range(len(indxIncl))]
indxs

In [ ]:
type(hsicRBF_jax(cube[smpl,indxs[1]],cube[smpl,indxs[0]][:,None]))

In [ ]:
type(sqeuclidean_distance(cube[0,:], cube[1,:]))

In [ ]:
type(cube[0,:])

In [ ]:
hsicsVars = np.array([[ float(hsicRBF_jax(cube[smpl,indxs[k]],cube[smpl,indxs[j]][:,None]))  for k in range(len(indxs))] for j in range(len(indxs))])

In [ ]:
for i in range(hsicsVars.shape[0]):
    hsicsVars[i,i] = np.nan

### Get figure 4 from paper

In [ ]:
fig, ax = plt.subplots()
im= ax.imshow(hsicsVars)
fig.colorbar(im)
plt.xticks(rotation=90)
plt.xticks(size=12)
plt.yticks(size=12)
ax.xaxis.set(ticks=np.arange(0, len(indxIncl)), ticklabels=np.array(varss)[indxIncl])
ax.yaxis.set(ticks=np.arange(0, len(indxIncl)), ticklabels=np.array(varss)[indxIncl])
#plt.savefig(ML_ready_root+"results/hsic_12.png", format='png', dpi=300)

In [ ]:
sns.distplot(hsicsVars)

### Get clusters

In [ ]:
def depVars(x):
    indx, = np.where(x>0.25)
    return indx

depGrps = [[i]+depVars(hsicsVars[i,:]).tolist() for i in range(hsicsVars.shape[0])]

In [ ]:
depGrps

In [ ]:
set(depGrps[2]).issubset(depGrps[7])

In [ ]:
i = 0
j = list(set(np.arange(hsicsVars.shape[0]).tolist()).difference([i]))[0]
indxKeep, = np.where([np.all(np.logical_not([set(depGrps[i]).issubset(depGrps[j]) for j in list(set(np.arange(hsicsVars.shape[0]).tolist()).difference([i]))])) for i in range(hsicsVars.shape[0])])
indxKeep

In [ ]:
[depGrps[i] for i in indxKeep]

In [ ]:
dp.flatten([depGrps[i] for i in indxKeep])

In [ ]:
#depGrps = [depGrps[i] for i in indxKeep]
len(depGrps)

In [ ]:
[np.array(varss)[indxIncl][d] for d in depGrps]

In [ ]:
indxIncl

In [ ]:
combosDep = dp.flatten([list(combinations(depGrps, i)) for i in range(1,12)])

In [ ]:
len(combosDep)

In [ ]:
combosDep = [np.array(varss)[indxIncl][np.sort(np.unique(np.concatenate(list(c))))].tolist() for c in combosDep]

In [ ]:
combosDep = np.unique(['-'.join(c) for c in combosDep]).tolist()

In [ ]:
combosDep = [c.split("-") for c in combosDep]

In [ ]:
len(combosDep)

In [ ]:
combosDep

In [ ]:
cluster_var

### exhaustive ICP based on variables in clusters

In [ ]:
start_time = time.time()
#res_ex2 = exhaustiveICP(cube, labels, envVars[:,0:3], event_df, cluster_var, varss, combosDep, posts, numTrees=100)
elapsed_time = time.time() - start_time
print("elapsed time: ", elapsed_time) # for 5 folds, 1 rep each



In [ ]:
results_root = "/home/emiliano/Documents/FDL/clean_code/results/"
version = "v_1Tail_0"
file = "exhaustiveICP_8_11_grouped"
#file = "exhaustiveICP_1_12_intersecRF_grouped"
fileRes = results_root+file+"_"+version+"_pvals.pkl" 

In [ ]:
#with open(fileRes, 'wb') as output:  # Overwrites any existing file.
#        pickle.dump(res_ex2, output, pickle.HIGHEST_PROTOCOL)


In [ ]:
res_ex2 = pickle5.load( open( fileRes, "rb" ) )

In [ ]:
len(res_ex2)

In [ ]:
sig = 0.05
indxCombos2, = np.where([ (r["pval_1tail"]>sig) for r in res_ex2])

In [ ]:
[res_ex2[i] for i in indxCombos2]

In [ ]:
type(res_ex2)

In [ ]:
indxCombos2

In [ ]:
combosPass2 = [combosDep[i] for i in indxCombos2]
len(combosPass2)

In [ ]:
combosPass2

In [ ]:
cmb2 = combosPass2[0]
for i in range(1,len(combosPass2)):
    cmb2 = set(cmb2).intersection(combosPass2[i])
list(cmb2)

In [ ]:
findIndxAux(varss, ["sshf"], 0)

In [ ]:
findIndxAux(varss, ["cape"], 0)

In [ ]:
depGrps

In [ ]:
k = 0
j = 0
combosPass2_grp = [list(np.where([set(depGrps[k]).issubset([findIndxAux(np.array(varss)[indxIncl],combosPass2[j],i) for i in range(len(combosPass2[j]))]) for k in range(len(depGrps))]))[0].tolist() for j in range(len(combosPass2))]

In [ ]:
cmb2_grp = combosPass2_grp[0]
for i in range(1,len(combosPass2_grp)):
    cmb2_grp = set(cmb2_grp).intersection(combosPass2_grp[i])
list(cmb2_grp)

In [ ]:
[depGrps[c] for c in list(cmb2_grp)]

In [ ]:
[np.array(varss)[indxIncl][d] for d in list(cmb2_grp)]

In [ ]:
[np.array(varss)[indxIncl][d] for d in [depGrps[c] for c in list(cmb2_grp)]]

In [ ]:
np.array(varss)[res["indxVar"]]

## Validation of greedyICP causal ordering using regional cross-validation (Figure 3 from paper)

In [ ]:
# impurity RF order
ordVars_RF = ['cvl','typeL','cvh','typeH','w','cin','ch2','sshf','u','slhf','uv250','blh','uv10','r750','r850','v','fg10',
 'ch1', 'ch3','ch4','u10','z','v10','alt','r650','ch5','ch6','cape']

# greedy-ICP order
ordVars_ICP = ['v10', 'cin', 'ch3', 'uv250', 'ch2', 'ch5', 'slhf', 'r750', 'fg10',
       'r650', 'u', 'ch4', 'cvh', 'typeL', 'typeH', 'cvl', 'w', 'blh',
       'uv10', 'z', 'ch1', 'u10', 'sshf', 'r850', 'v', 'ch6', 'cape',
       'alt']

In [ ]:
i = 1
unions = [list(set(ordVars_RF[::-1][0:i]).union(ordVars_ICP[::-1][0:i])) for i in range(1, len(ordVars_RF))]

In [ ]:
[len(u) for u in unions]

In [ ]:
np.sum([comb(12,i) for i in range(12)])

In [ ]:
len(unions[7])
unions[7]

In [ ]:
ordVars = ordVars_ICP

In [ ]:
combosOrd = [ordVars[i:len(ordVars)] for i in range(len(ordVars))]

In [ ]:
len(ordVars)
len(combosOrd[0])

In [ ]:
cluster_var

In [ ]:
combosOrd

In [ ]:
from rf import *

In [ ]:
np.random.seed(1234556)
num_reps = 1
seed = 1234
res = [getFold(cluster, cluster_var, cube, labels, event_df, num_reps, seed, byinistate=False, importance=False, shuffle=True) for cluster in np.unique(event_df[cluster_var])]

In [ ]:
def maxAUC(x):
    indx = np.argmax(x.auc)
    #print(indx)
    return x.iloc[indx]

In [ ]:
res_msrs = [list(rm)[0] for rm in res]
res_msrs = pd.concat(res_msrs)
resMax = res_msrs[["rep","fold","auc","fpr","fnr"]].groupby(["fold"]).apply(maxAUC)#.reset_index()#.rename(columns={0:"auc"})
res = {"mean":np.mean(resMax.auc), "std":np.std(resMax.auc)}
print(res)
# for event {'mean': 0.5444933526439506, 'std': 0.070835034298292}
# for spatial {'mean': 0.4433585660768273, 'std': 0.07028088333880186}

In [ ]:
posts =np.arange(0,cube.shape[1]+1-10, 11)
posts = np.array(posts.tolist()+[posts[posts.shape[0]-1]+4,posts[posts.shape[0]-1]+10])
posts

In [ ]:
cluster_var

In [ ]:
num_reps = 1
seed = 1234

start_time = time.time()
#resSeqICP = [getSeqICP(c, varss,posts, "cluster_regional", cube, labels, event_df, num_reps, seed) for c in combosOrd]
elapsed_time = time.time() - start_time
print("elapsed time: ", elapsed_time) # for 5 folds, 1 rep each



In [ ]:
results_root = "/home/emiliano/Documents/FDL/clean_code/results/"

file = "rf_pred_era5_19vs_29vs"


version = "vRegion_ICP_selected"
#version = "vRand_ICP_selected"

fileRes = results_root+file+"_"+version+"ICPseq.png"
#fileRes = results_root+file+"_"+version+"RFimpurity_seq.png"
fileRes

In [ ]:
#with open(fileRes, 'wb') as output:  # Overwrites any existing file.
#        pickle.dump(resSeqICP, output, pickle.HIGHEST_PROTOCOL)


In [ ]:
resSeqICP[0]

In [ ]:
resICP_mean = [r["mean"] for r in resSeqICP]
resICP_std = [r["std"] for r in resSeqICP]

In [ ]:
plt.scatter(ordVars, resICP_mean)

In [ ]:
plt.scatter(ordVars, resICP_mean)

In [ ]:
plt.scatter(ordVars, resICP_std)

In [ ]:
plt.scatter(ordVars, resICP_std)

In [ ]:
len(ordVars)

In [ ]:
file

In [ ]:

# greedy-ICP order
ordVars = ['v10', 'cin', 'ch3', 'uv250', 'ch2', 'ch5', 'slhf', 'r750', 'fg10',
       'r650', 'u', 'ch4', 'cvh', 'typeL', 'typeH', 'cvl', 'w', 'blh',
       'uv10', 'z', 'ch1', 'u10', 'sshf', 'r850', 'v', 'ch6', 'cape',
       'alt']

results_root = "/home/emiliano/Documents/FDL/clean_code/results/"
file = "rf_pred_era5_19vs_29vs"

version = "vRand_ICP_selected"
fileRes = results_root+file+"_"+version+"ICPseq.pkl"

resSeqICPrand = pickle5.load( open( fileRes, "rb" ) )
resICPrand_mean = [r["mean"] for r in resSeqICPrand]
resICPrand_std = [r["std"] for r in resSeqICPrand]
df_rand = pd.DataFrame({"var":ordVars,"mean_AUC":resICPrand_mean, "std_AUC":resICPrand_std})
df_rand["CV"] = "random"
df_rand["order"] = "ICP"
df_rand["idx_var"] = np.arange(df_rand.shape[0])

In [ ]:
results_root = "/home/emiliano/Documents/FDL/clean_code/results/"
file = "rf_pred_era5_19vs_29vs"
version = "vRegion_ICP_selected"

fileRes = results_root+file+"_"+version+"ICPseq.pkl"

resSeqICPreg = pickle5.load( open( fileRes, "rb" ) )
resICPreg_mean = [r["mean"] for r in resSeqICPreg]
resICPreg_std = [r["std"] for r in resSeqICPreg]
df_region = pd.DataFrame({"var":ordVars,"mean_AUC":resICPreg_mean, "std_AUC":resICPreg_std})
df_region["CV"] = "regional"
df_region["order"] = "ICP"
df_region["idx_var"] = np.arange(df_region.shape[0])

In [ ]:
df = pd.concat([df_rand, df_region])
df = pd.melt(df, id_vars=["var","CV","order","idx_var"])
df.head()

In [ ]:
df = pd.concat([df_rand, df_region])
df = pd.melt(df, id_vars=["var","CV","order","idx_var"])
df.head()

In [ ]:
file

In [ ]:
version1 = "ICP_selected"

In [ ]:
fileRes = results_root+file+"_"+version1+"valid_AUC_mean_std.png"
fileRes

In [ ]:
g = sns.FacetGrid(df, col="variable",hue="CV",margin_titles=True, sharey=False)
g.map(sns.stripplot, "var", "value",  size=7)
#g.set(xlabel='variable excluded', ylabel='auc')
g.set_titles(col_template="{col_name}",size=16)
#g.set_titles(row_template="{row_name}",size=16)

    #g.set(xticks=onp.unique(resAux["num"]))
    #g.set_xticklabels(par_vals)

g.set_xticklabels(rotation=90)
g.set_xticklabels(size=10)


g.set_axis_labels("Seq. of excluded vars.", "auc", size=16)

g.add_legend()
#plt.legend(title='Team', fontsize='10', title_fontsize='14')

plt.setp(g._legend.get_title(), fontsize=20)
plt.setp(g._legend.get_texts(), fontsize=15)

#plt.legend(bbox_to_anchor=(1.3, 1),borderaxespad=0)
#g.savefig(fileRes, format='png', dpi=300)
    
    